In [1]:
# adjust the R limit
memory.limit(size= 81011630)

[1] 81011630

In [13]:
#necessary library
library(data.table) #fread
library(lubridate)
library(reshape2)
library(dplyr)
library(scales)
library(stringr)
#library(tidyverse)

Warning message:
"package 'data.table' was built under R version 3.6.3"

Attaching package: 'lubridate'


The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following object is masked from 'package:base':

    date


Warning message:
"package 'reshape2' was built under R version 3.6.3"

Attaching package: 'reshape2'


The following objects are masked from 'package:data.table':

    dcast, melt



Attaching package: 'dplyr'


The following objects are masked from 'package:lubridate':

    intersect, setdiff, union


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




## Bin speeds and aggregate table

In [2]:
interval_df = read.csv("../../data/tidy/train-trajectory-19.csv")
dc = fread("../../data/raw/energy-consumption-08-19.csv") # Read in energy data

ERROR: Error: cannot allocate vector of size 1000.0 Mb


In [3]:
interval_df = read.csv("../../data/raw/test.csv")

In [22]:
# select random days
set.seed(1234)
sub = sample(1:nrow(interval_df),720)
interval_df_test = interval_df[sub,]

In [10]:
# Unit conversion
unit_transfer = function(df){
   df$speed_mph = df$speed_kph*0.621371 #kph to mph
   df$distance_mile = df$dist_meters*0.000621371 #convert from meters to mile
   df$time_hr = df$interval_seconds/3600.0 #convert from seconds to hour
  return(df)
}    

In [11]:
interval_df = unit_transfer(interval_df)

In [35]:
interval_df = data.table(interval_df )

In [59]:
head(interval_df)

trxtime,trainid,lineid,lat,lon,day,month,year,dist_meters,interval_seconds,...,accel_bin_1_dummy,accel_bin_2_dummy,accel_bin_3_dummy,accel_bin_4_dummy,accel_bin_5_dummy,accel_bin_1_time_hr,accel_bin_2_time_hr,accel_bin_3_time_hr,accel_bin_4_time_hr,accel_bin_5_time_hr
<fct>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2019-01-01 04:00:00,10001,4,42.33701,-71.25349,1,1,2019,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2019-01-01 04:42:35,10001,4,42.33613,-71.25391,1,1,2019,104.64023,2555,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2019-01-01 04:45:05,10001,4,42.33603,-71.25436,1,1,2019,38.35157,150,...,0,0,1,0,0,0,0,0.04166667,0.000000000,0
2019-01-01 05:49:31,10001,4,42.33613,-71.25391,1,1,2019,38.35157,3866,...,0,0,1,0,0,0,0,1.07388889,0.000000000,0
2019-01-01 05:49:42,10001,4,42.33635,-71.25379,1,1,2019,26.23150,11,...,0,0,0,1,0,0,0,0.00000000,0.003055556,0
2019-01-01 06:19:33,10001,4,42.33658,-71.25367,1,1,2019,28.08000,1791,...,0,0,1,0,0,0,0,0.49750000,0.000000000,0


In [53]:
# Calculate the speed bins 
bin_speeds <- function (dataframe, num_bins) {
  dummy_cols = c(paste0("speed_bin_",1:num_bins,"_dummy"))
  bin_time_cols = c(paste0("speed_bin_",1:num_bins,"_time_hr"))
  cutpoints <- quantile(dataframe$speed_mph,seq(0, 1, 1/num_bins),na.rm=TRUE) 
  print(paste0("The speed bins are: ", cutpoints))
  for(n in seq(1, num_bins)) {
    if(n == 1){
     dataframe[, dummy_cols[n] := sapply(speed_mph,  function(x) ifelse (x < cutpoints[n+1], 1, 0))]
    }
    else if (n == num_bins){
     dataframe[, dummy_cols[n] := sapply(speed_mph,  function(x) ifelse (x >= cutpoints[num_bins], 1, 0))]
    }
    else {
    dataframe[, dummy_cols[n] := sapply(speed_mph,  function(x) ifelse (x >= cutpoints[n] & x < cutpoints[n + 1], 1, 0))]
    }
  }
  dataframe[, bin_time_cols := lapply(.SD, function(x) x * dataframe$time_hr ), .SDcols = dummy_cols]
  print(paste0("Percentage error of summed speed bin times = ", 
               round(100*(sum(colSums(dataframe[, bin_time_cols],na.rm=TRUE)) 
                          - sum(dataframe$time_hr, na.rm=TRUE))/sum(dataframe$time_hr, na.rm=TRUE),2), "%"))                                                  
  return(dataframe, cutpoints)
}

In [54]:
bin_speeds(interval_df,5)

[1] "The speed bins are: 0"               
[2] "The speed bins are: 9.23266801722276"
[3] "The speed bins are: 21.7523583954108"
[4] "The speed bins are: 23.7228098677007"
[5] "The speed bins are: 45.2297682434496"
[6] "The speed bins are: 6079.42977146183"


ERROR: Error in `[.data.table`(dataframe, , `:=`(bin_time_cols, lapply(.SD, function(x) x * : Supplied 5 items to be assigned to 2000 items of column 'bin_time_cols'. If you wish to 'recycle' the RHS please use rep() to make this intent clear to readers of your code.


In [57]:
bin_accelerations(interval_df,5)

[1] "The acceleration bins are: -437.187112719642"  
[2] "The acceleration bins are: -0.759663458424666" 
[3] "The acceleration bins are: -0.0821155100901342"
[4] "The acceleration bins are: 0.0707228569514413" 
[5] "The acceleration bins are: 1.41554942883802"   
[6] "The acceleration bins are: 2695.87057916446"   


ERROR: Error in `[.data.table`(dataframe, , bin_time_cols): j (the 2nd argument inside [...]) is a single symbol but column name 'bin_time_cols' is not found. Perhaps you intended DT[, ..bin_time_cols]. This difference to data.frame is deliberate and explained in FAQ 1.1.


In [56]:
# Calculate the acceleration bins 
bin_accelerations <- function (dataframe, num_bins) {
  dummy_cols = c(paste0("accel_bin_",1:num_bins,"_dummy"))
  bin_time_cols = c(paste0("accel_bin_",1:num_bins,"_time_hr"))
  cutpoints <- quantile(dataframe$accel_mps2,seq(0, 1, 1/num_bins),na.rm=TRUE) 
  print(paste0("The acceleration bins are: ", cutpoints))
  for(n in seq(1, num_bins)) {
    if(n == 1){
      dataframe[, dummy_cols[n] := sapply(accel_mps2,  function(x) ifelse (x < cutpoints[n+1], 1, 0))]
    }
    else if (n == num_bins){
      dataframe[, dummy_cols[n] := sapply(accel_mps2,  function(x) ifelse (x >= cutpoints[num_bins], 1, 0))]
    }
    else {
      dataframe[, dummy_cols[n] := sapply(accel_mps2,  function(x) ifelse (x >= cutpoints[n] & x < cutpoints[n + 1], 1, 0))]
    }
  }
  dataframe[, (bin_time_cols) := lapply(.SD, function(x) x * dataframe$time_hr ), .SDcols = dummy_cols]
  print(paste0("Percentage error of summed acceleration bin times = ", 
               round(100*(sum(colSums(dataframe[, bin_time_cols],na.rm=TRUE)) 
                          - sum(dataframe$time_hr, na.rm=TRUE))/sum(dataframe$time_hr, na.rm=TRUE),2), "%"))                                                  
  return(dataframe, cutpoints)
}

In [12]:
# Aggregate dataframe at hour level
hour_aggregate <- function (dataframe) {
    sum_cols = c("distance_mile","time_hr",paste0("speed_bin_",1:5,"_time_hr"),paste0("accel_bin_",1:5,"_time_hr"))
    agg_df = dataframe[, lapply( .SD, sum , na.rm=TRUE), by = c('Month','Hour','Day'), .SDcols = sum_cols]
    avg_interval_speed_mph_df = dataframe[, lapply( .SD, mean , na.rm=TRUE), by = c('Month','Hour','Day'), .SDcols = 'speed_mph']
    agg_df[, 'avg_interval_speed_mph'] = avg_interval_speed_mph_df$speed_mph
    agg_df[, 'avg_hour_speed_mph'] = agg_df$distance_mile/agg_df$time_hr
    return(agg_df)
}

In [13]:
# Combine energy consumption data
merge_energy <- function (hour_dt, energy_df) {    
    # Melt by hour 
    melted_energy_df = melt(energy_df, id.vars=c('Year','Month', 'Day of Month','WJ','TAVG'),measure.vars = paste0("Hour ", 1:24))
    colnames(melted_energy_df) = c('Year', 'Month', 'Day', 'WJ', 'TAVG', 'Hour', 'energy_kwh')
    hour_energy_dt <- setDT(melted_energy_df)
    hour_energy_dt[, Hour := str_replace(Hour, "Hour ", "")]
    hour_energy_dt$Hour = as.numeric(hour_energy_dt$Hour)
    hour_energy_dt$Hour = hour_energy_dt$Hour - 1
    merged_dt = merge(hour_dt, hour_energy_dt, all=T)
    return(merged_dt)
}

## Computing the final table

In [14]:
# Select one year for enenrgy data extraction
energy2019 = subset(dc,dc$Year==2019)

In [15]:
main <- function (movements_df, energy_df) {
    final_df <- movements_df %>% bin_speeds() %>% bin_accelerations() %>% hour_aggregate() %>% merge_energy(energy_df)
    return(final_df)
}

In [ ]:
df_final = main(interval_df,energy2019)
write.csv(df_final,"../Data/tidy/system-model.csv")

# Experimental section

In [ ]:
# # Calculate the acceleration bins
# bin_accelerations <- function (dataframe) {
#     dummy_cols = c(paste0("accel_bin_",1:5,"_dummy"))
#     bin_time_cols = c(paste0("speed_bin_",1:5,"_time_hr"))
#     cutpoints  <- quantile(dataframe$accel_mps2,seq(0,1,.2),na.rm=TRUE) 
#     print(paste0("The acceleration bins are: ", testcuts))
#     dataframe[, 'accel_bin_1_dummy' := sapply(accel_mps2,  function(x) ifelse (x < testcuts[2], 1, 0))]
#     dataframe[, 'accel_bin_2_dummy' := sapply(accel_mps2,  function(x) ifelse (x >= testcuts[2] & x < testcuts[3], 1, 0))]
#     dataframe[, 'accel_bin_3_dummy' := sapply(accel_mps2,  function(x) ifelse (x >= testcuts[3] & x < testcuts[4], 1, 0))]
#     dataframe[, 'accel_bin_4_dummy' := sapply(accel_mps2,  function(x) ifelse (x >= testcuts[4] & x < testcuts[5], 1, 0))]
#     dataframe[, 'accel_bin_5_dummy' := sapply(accel_mps2,  function(x) ifelse (x >= testcuts[5], 1, 0))]
#     dataframe[, (bin_time_cols) := lapply(.SD, function(x) x * dataframe$time_hr ), .SDcols = dummy_cols]
#     print(paste0("Percentage error of summed acceleration bin times = ", round(100*(sum(colSums(dataframe[, bin_time_cols ],na.rm=TRUE)) 
#                - sum(dataframe$time_hr, na.rm=TRUE))/sum(dataframe$time_hr, na.rm=TRUE),2), "%"))                                                  
#     return(dataframe)
# }